# 理论介绍

上下文块头 （CCH） 是一种创建包含更高级别上下文（例如文档级或章节级上下文）的块头的方法，并在嵌入这些块头之前将这些块头附加到块中。这为嵌入提供了文本内容和含义的更准确和完整的表示。在我们的测试中，此功能可显著提高检索质量。除了提高检索正确信息的速度外，CCH 还降低了不相关结果在搜索结果中的显示速度。这降低了 LLM 在下游聊天和生成应用程序中误解一段文本的速率。

开发人员在使用 RAG 时面临的许多问题都归结为：单个块通常不包含足够的上下文，无法被检索系统或 LLM 正确使用。这导致无法回答问题，更令人担忧的是，还会出现幻觉。具体场景有
- Chunk 通常通过隐含的引用和代词来指代其主题。这会导致它们在应该检索的时候没有被检索，或者 LLM 无法正确理解它们。
- 单个块通常只在整个部分或文档的上下文中才有意义，并且单独阅读时可能会产生误导。

实现步骤
1. 上下文生成：使用 LLM 为文档生成描述性标题。具体实现为：利用LLM完成简单的prompt模版，对每一个chunk生成描述性标题。如果有足够描述性的文档标题，则可以直接使用这些标题。例如：简明的文档摘要、章节/子章节标题。
2. 将生成chunk header 嵌入 chunk
3. 在结果中返回chunk header

# 代码实现

## 数据准备

In [1]:
import re
import json
import tiktoken
import os 

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.chroma import Chroma
from openai import OpenAI
from dotenv import load_dotenv
from tqdm import tqdm

from langchain_openai import ChatOpenAI
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain import PromptTemplate

/Users/jinzhang/Desktop/C7 高级 RAG 技巧/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY') # OpenAI API key

pdf_path = "data/pumpkin_book.pdf"
qa_path = 'data/train_dataset.json'
embedding = HuggingFaceEmbeddings(model_name='BAAI/bge-small-zh-v1.5')

with open(qa_path, 'r', encoding='utf-8') as file:
    qa_pairs = json.load(file)

/var/folders/h7/ft2ksmt92hd5vbtf1kj60wqc0000gn/T/ipykernel_5077/3694944193.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name='BAAI/bge-small-zh-v1.5')
/Users/jinzhang/Desktop/C7 高级 RAG 技巧/.venv/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Constants
DOCUMENT_TITLE_PROMPT = """
指令
总结以下文档内容的标题是什么？

您的回答直接输出内容标题且仅此而已。请不要回应其他内容。

{document_title_guidance}

{truncation_message}

文档
{document_text}
""".strip()

TRUNCATION_MESSAGE = """
请注意，下面提供的文档文本仅为文档的前~{num_words}个词。这对于此任务来说已经足够。您的回答仍应与整个文档相关，而不仅仅是下面提供的文本。
""".strip()

MAX_CONTENT_TOKENS = 4000
MODEL_NAME = "gpt-4o-mini"
TOKEN_ENCODER = tiktoken.encoding_for_model('gpt-3.5-turbo')

## 实现示例

In [4]:
def clean_text(text: str):
    """
    实现文本清理函数

    参数:
        text: 需要清理的字段

    返回:
        清理完成后返回的字段
    """
    # 删除每页开头与结尾标语及链接
    text = re.sub(r'→_→\n欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》\n←_←', '', text)
    text = re.sub(r'→_→\n配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU\n←_←', '', text)
    # 删除字符串开头的空格
    text = re.sub(r'\s+', '', text)
    # 删除回车
    text = re.sub(r'\n+', '', text)

    return text

def make_llm_call(chat_messages: list[dict]) -> str:
    """
    调用 OpenAI 语言模型的 API。

    参数:
        chat_messages (list[dict]): 用于聊天完成的消息字典列表。

    返回:
        str: 语言模型生成的响应。
    """
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=chat_messages,
        max_tokens=MAX_CONTENT_TOKENS,
        temperature=0.2,
    )
    return response.choices[0].message.content.strip()


def truncate_content(content: str, max_tokens: int) -> tuple[str, int]:
    """
    将内容截断为指定的最大token数。

    参数:
        content (str): 需要截断的输入文本。
        max_tokens (int): 保留的最大token数。

    返回:
        tuple[str, int]: 包含截断后内容和令牌数量的元组。
    """
    tokens = TOKEN_ENCODER.encode(content, disallowed_special=())
    truncated_tokens = tokens[:max_tokens]
    return TOKEN_ENCODER.decode(truncated_tokens), min(len(tokens), max_tokens)

def get_document_title(document_text: str, document_title_guidance: str = "") -> str:
    """
    使用语言模型提取文档标题。

    参数:
        document_text (str): 文档的文本内容。
        document_title_guidance (str, 可选): 提取标题的额外指导。默认为 ""。

    返回:
        str: 提取的文档标题。
    """
    document_text, num_tokens = truncate_content(document_text, MAX_CONTENT_TOKENS)
    truncation_message = TRUNCATION_MESSAGE.format(num_words=3000) if num_tokens >= MAX_CONTENT_TOKENS else ""

    prompt = DOCUMENT_TITLE_PROMPT.format(
        document_title_guidance=document_title_guidance,
        document_text=document_text,
        truncation_message=truncation_message
    )
    chat_messages = [{"role": "user", "content": prompt}]
    
    return make_llm_call(chat_messages)

In [5]:
example_text = '“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读者通过西瓜书对机器学习有所了解， 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推导细节的读者'

document_title = get_document_title(example_text)
print(f"Document Title: {document_title}")

Document Title: 《机器学习》入门教材总结


## 功能实现

In [6]:
def encode_pdf(path, chunk_size=2000, chunk_overlap=100, is_header=False):
    """
    使用 OpenAI 嵌入将 PDF 书籍编码为向量存储。

    参数:
        path: PDF 文件的路径。
        chunk_size: 每个文本块的期望大小。
        chunk_overlap: 连续块之间的重叠量。

    返回:
        包含内容的向量存储。
    """

    # 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径，加载PDF
    loader = PyMuPDFLoader(path)
    
    # 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
    pdf_pages = loader.load()
    
    # 第13页为南瓜书第一页正文，因此从13页开始,从倒数13页涉及敏感用语，因此从-13页结束
    data_pages = pdf_pages[13:-13]

    for page in data_pages:
        page.page_content = clean_text(page.page_content)

    # 文档分块
    text_splitter = CharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, separator='')

    split_docs = text_splitter.split_documents(data_pages)

    # 给每一个分块增加header
    if is_header:
        for split_doc in tqdm(split_docs):
            document_title = get_document_title(split_doc.page_content)
            split_doc.page_content = f"文章标题: {document_title}\n\n{split_doc.page_content}"

    # 构建向量库
    vectordb = Chroma.from_documents(documents=split_docs, embedding=embedding)

    return vectordb

In [7]:
class CCHRetriever:
    def __init__(self, chunk_size=2000, chunk_overlap=100):
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)
        self.embeddings = embedding
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

    # 基于pdf构建向量数据库   
    # 构建不包含header的db 
    def encode_pdf_to_vectorstore_wo_header(self, files_path):
        self.vectorstore_wo_header = encode_pdf(files_path, chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap, is_header=False)
    # 构建包含header的db
    def encode_pdf_to_vectorstore_w_header(self, files_path):
        self.vectorstore_w_header = encode_pdf(files_path, chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap, is_header=True)

In [8]:
retriever = CCHRetriever()
retriever.encode_pdf_to_vectorstore_wo_header(pdf_path)
retriever.encode_pdf_to_vectorstore_w_header(pdf_path)

100%|██████████| 170/170 [04:49<00:00,  1.70s/it]


In [10]:
test_doc = 1

qa_pairs[test_doc]

{'query': '请根据提供的上下文信息，解释什么是“泛化”能力，并给出一个具体的例子说明为何泛化能力是衡量机器学习模型好坏的关键。',
 'answer': '泛化：由于机器学习的目标是根据已知来对未知做出尽可能准确的判断，因此对未知事物判断的准确与否才是衡量一个模型好坏的关键，我们称此为“泛化”能力。例如学习西瓜好坏时，假设训练集中共有3个样本：{(x1=(青绿;蜷缩),y1=好瓜),(x2=(乌黑;蜷缩),y2=好瓜),(x3=(浅白;蜷缩),y3=好瓜)}，同时假设判断西瓜好坏的真相是“只要根蒂蜷缩就是好瓜”，如果应用算法A在此训练集上训练得到模型fa(x)，模型a学到的规律是“色泽等于青绿、乌黑或者浅白时，同时根蒂蜷缩即为好瓜，否则便是坏瓜”，再应用算法B在此训练集上训练得到模型fb(x)，模型fb(x)学到的规律是“只要根蒂蜷缩就是好瓜”，因此对于一个未见过的西瓜样本x=(金黄;蜷缩)来说，模型fa(x)给出的预测结果为“坏瓜”，模型fb(x)给出的预测结果为“好瓜”，此时我们称模型fb(x)的泛化能力优于模型fa(x)。通过以上举例可知，尽管模型fa(x)和模型fb(x)对训练集学得一样好，即两个模型对训练集中每个样本的判断都对，但是其所学到的规律是不同的。导致此现象最直接的原因是算法的不同，但是算法通常是有限的，可穷举的，尤其是在特定任务场景下可使用的算法更是有限，因此，数据便是导致此现象的另一重要原因，这也就是机器学习领域常说的“数据决定模型的上限，而算法则是让模型无限逼近上限”。',
 'page_num': 14}

In [11]:
test_query = qa_pairs[test_doc]['query']
results = retriever.vectorstore_wo_header.similarity_search(test_query, k=5)
results


[Document(metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': 'data/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 17, 'producer': 'xdvipdfmx (20200315)', 'source': 'data/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='第2章模型评估与选择如“西瓜书”前言所述，本章仍属于机器学习基础知识，如果说第1章介绍了什么是机器学习及机器学习的相关数学符号，那么本章则进一步介绍机器学习的相关概念。具体来说，介绍内容正如本章名称“模型评估与选择”所述，讲述的是如何评估模型的优劣和选择最适合自己业务场景的模型。由于“模型评估与选择”是在模型产出以后进行的下游工作，要想完全吸收本章内容需要读者对模型有一些基本的认知，因此零基础的读者直接看本章会很吃力，实属正常，在此建议零基础的读者可以简单泛读本章，仅看能看懂的部分即可，或者直接跳过本章从第3章开始看，直至看完第6章以后再回头来看本章便会轻松许多。2.1经验误差与过拟合梳理本节的几个概念。错误率：E=am，其中m为样本个数，a为分类错误样本个数。精度：精度=1-错误率。误差：学习器的实际预测输出与样本的真实输出之间的差异。经验误差：学习器在训练集上的误差，又称为“训练误差”。泛化误差：学习器在新样本上的误差。经验误差和泛化误差用于分类问题的定义式可参见“西瓜书”第12章的式(12.1)和式(12.2)，接下来辨析一下以上几个概念。错误率和精度很容易理解，而且很明显是针对分类问题的。误差的概念更适用于回归问题，但是，根据“西瓜书”第12章的式(12.1)和式(12.2)的定义可以看出，在分类问题中也会使用误差的概念，此时的“差异”指的是学习器的实际预测输出的类别与样本真实的类别是否一致，若一致则“差异”为0，若不一致则“差异”为1，训练误差是

In [12]:
test_query = qa_pairs[test_doc]['query']
results = retriever.vectorstore_w_header.similarity_search(test_query, k=5)
results

[Document(metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': 'data/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 17, 'producer': 'xdvipdfmx (20200315)', 'source': 'data/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='第2章模型评估与选择如“西瓜书”前言所述，本章仍属于机器学习基础知识，如果说第1章介绍了什么是机器学习及机器学习的相关数学符号，那么本章则进一步介绍机器学习的相关概念。具体来说，介绍内容正如本章名称“模型评估与选择”所述，讲述的是如何评估模型的优劣和选择最适合自己业务场景的模型。由于“模型评估与选择”是在模型产出以后进行的下游工作，要想完全吸收本章内容需要读者对模型有一些基本的认知，因此零基础的读者直接看本章会很吃力，实属正常，在此建议零基础的读者可以简单泛读本章，仅看能看懂的部分即可，或者直接跳过本章从第3章开始看，直至看完第6章以后再回头来看本章便会轻松许多。2.1经验误差与过拟合梳理本节的几个概念。错误率：E=am，其中m为样本个数，a为分类错误样本个数。精度：精度=1-错误率。误差：学习器的实际预测输出与样本的真实输出之间的差异。经验误差：学习器在训练集上的误差，又称为“训练误差”。泛化误差：学习器在新样本上的误差。经验误差和泛化误差用于分类问题的定义式可参见“西瓜书”第12章的式(12.1)和式(12.2)，接下来辨析一下以上几个概念。错误率和精度很容易理解，而且很明显是针对分类问题的。误差的概念更适用于回归问题，但是，根据“西瓜书”第12章的式(12.1)和式(12.2)的定义可以看出，在分类问题中也会使用误差的概念，此时的“差异”指的是学习器的实际预测输出的类别与样本真实的类别是否一致，若一致则“差异”为0，若不一致则“差异”为1，训练误差是

## 效果测评

In [45]:
# 不使用CCH
from tqdm import tqdm

i = 0
j = 0
for qa_pair in tqdm(qa_pairs):
    if len(qa_pair['query']) > 10:
        query = qa_pair['query']
        sim_docs = retriever.vectorstore_wo_header.similarity_search(query, k=10)
        page_nums = [doc.metadata['page'] for doc in sim_docs]
        if qa_pair['page_num'] in page_nums: i += 1
        j += 1

100%|██████████| 119/119 [00:02<00:00, 39.83it/s]


In [46]:
print(f"召回率为: {i/j * 100}%")

召回率为: 83.63636363636363%


In [47]:
# 不使用CCH
from tqdm import tqdm

i = 0
j = 0
for qa_pair in tqdm(qa_pairs):
    if len(qa_pair['query']) > 10:
        query = qa_pair['query']
        sim_docs = retriever.vectorstore_w_header.similarity_search(query, k=10)
        page_nums = [doc.metadata['page'] for doc in sim_docs]
        if qa_pair['page_num'] in page_nums: i += 1
        j += 1

100%|██████████| 119/119 [00:02<00:00, 42.01it/s]


In [48]:
print(f"召回率为: {i/j * 100}%")

召回率为: 83.63636363636363%
